In [114]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import re
import numpy as np

In [18]:
price= []
mkmodel = []
mileage= []
year = []

In [19]:
url = "https://www.cars.com/shopping/results/?page=1&page_size=100&list_price_max=&makes[]=&maximum_distance=20&models[]=&stock_type=used&zip=53151"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
           "Accept-Encoding":"gzip, deflate",  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [140]:
def pagelooper(zip):
    getcount = requests.get(f"""https://www.cars.com/shopping/results/?page=1&page_size=100&list_price_max=&makes[]
    =&maximum_distance=20&models[]=&stock_type=used&zip={zip}""",headers=headers)
    setup1 = BeautifulSoup(getcount.content,"html.parser")
    setup2 = BeautifulSoup(setup1.prettify(),'html.parser')
    pagecounter = setup2.find(class_="total-filter-count").get_text()
    x =  ''.join(re.findall(r"\d+", pagecounter.strip()))
    x2 = int(x) // 100 -1
    print(f"{x2} pages to print...")
   
    for i in range(1,x2):
        while(1):
            try:
                page = requests.get(f"""https://www.cars.com/shopping/results/?page={i}&page_size=100&list_price_max=&makes[]
                =&maximum_distance=20&models[]=&stock_type=used&zip={zip}""",headers=headers)

                soup1 = BeautifulSoup(page.content,"html.parser")
                soup2= BeautifulSoup(soup1.prettify(),'html.parser')
                search = soup2.find_all(class_="primary-price")
                search2 = soup2.find_all(class_="title")
                search3 = soup2.find_all(class_='mileage')
            except AttributeError:
                print('Retrying')
                time.sleep(3)
            else:
                break
        for i,j,k in zip(search,search2,search3,):
            price.append(i.get_text().strip())
            year.append(j.get_text().strip()[0:4])
            mkmodel.append(j.get_text().strip()[5:])
            mileage.append(k.get_text().strip())
        time.sleep(2)
        print(f"Page {i} scanned...")
    print("Done!")

In [40]:
pagelooper()

Page 1 scanned...
Page 2 scanned...
Page 3 scanned...
Page 4 scanned...
Page 5 scanned...
Page 6 scanned...
Page 7 scanned...
Page 8 scanned...
Page 9 scanned...
Page 10 scanned...
Page 11 scanned...
Page 12 scanned...
Page 13 scanned...
Page 14 scanned...
Page 15 scanned...
Page 16 scanned...
Page 17 scanned...
Page 18 scanned...
Page 19 scanned...
Page 20 scanned...
Page 21 scanned...
Page 22 scanned...
Page 23 scanned...
Page 24 scanned...
Page 25 scanned...
Page 26 scanned...
Page 27 scanned...
Page 28 scanned...
Page 29 scanned...
Page 30 scanned...
Page 31 scanned...
Page 32 scanned...
Page 33 scanned...
Page 34 scanned...
Page 35 scanned...
Page 36 scanned...
Page 37 scanned...
Page 38 scanned...
Page 39 scanned...
Page 40 scanned...
Page 41 scanned...
Page 42 scanned...
Page 43 scanned...
Page 44 scanned...
Page 45 scanned...
Page 46 scanned...
Page 47 scanned...
Page 48 scanned...
Page 49 scanned...
Page 50 scanned...
Page 51 scanned...
Page 52 scanned...
Page 53 scanned...
Pa

In [141]:
def cardata():
    
    cardata= list(zip(price,year,mkmodel,mileage))
    need = pd.DataFrame(cardata,columns=['Price','Year','Make/Model','Mileage'])
    need['mileage'] = need['mileage'].str.replace(r'\D+','').astype(int)
    need.loc[need['price'] == 'Not Priced', 'price'] = '0'
    need['price'] = need['price'].str.replace(r"[$,]",'').astype(int)
    need['price'] = need['price'].replace(0,np.NaN)
    need['price'] = need['price'].astype('Int64')
    return need

In [136]:
cardata()

<ipython-input-134-5a2c7c270304>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  need['mileage'] = need['mileage'].str.replace(r'\D+','').astype(int)
<ipython-input-134-5a2c7c270304>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  need['price'] = need['price'].str.replace(r"[$,]",'').astype(int)


,price,year,mkmodel,mileage
0,38899,2019,Toyota RAV4 Hybrid Limited,15415
1,14311,2016,Mazda Mazda3 i Sport,75312
2,35995,2019,Toyota Tacoma TRD Sport,30418
3,46939,2018,BMW X5 xDrive35i,32775
4,29499,2018,Chevrolet Camaro 2LT,13616
...,...,...,...,...
8531,16984,2014,Jeep Cherokee Limited,94954
8532,38987,2018,Jeep Wrangler Unlimited Sport,34473
8533,39387,2017,Chevrolet Silverado 1500 LT,38486
8534,37987,2018,Chevrolet Silverado 1500 LT,69919


In [137]:
need.price.mean()

29601.94306466729

In [139]:
need.to_csv('carscom.csv', index=False)